# <center>Ch 7. DPO微调Qwen2.5 </center>

# 1. 什么是DPO

## 1.1 DPO的背景与意义

随着大语言模型（LLM）的发展，尤其是模型在自然语言生成任务中的应用逐渐增多，传统的监督学习方法已不足以满足这些模型的优化需求。为了提升模型的性能，研究人员开始引入强化学习（RL）进行微调，目前最著名的就是**RLHF**（Reinforcement Learning from Human Feedback，基于人类反馈的强化学习）。

### 为什么会产生 DPO（Direct Preference Optimization）算法？

DPO的出现，主要是为了克服传统RLHF方法的一些局限性和挑战。尽管RLHF在许多大规模语言模型中被广泛使用，取得了显著的成功，但它也存在一些问题，才会使得了DPO应运而生。我们可以从以下几个方面来理解为什么会产生DPO：

### 1. RLHF 的局限性
RLHF是通过**人工反馈**来训练模型的主要方法，它利用**人类的评价或排序来构建奖励模型**，然后使用强化学习技术（如PPO）来优化模型。然而，RLHF 面临以下几个挑战：
   - **高成本和时间消耗**：RLHF需要大量的人类反馈，尤其是在模型生成文本时，需要人类评审生成的内容并提供反馈。这一过程不仅费时，还需要大量的人工资源，对于大规模模型来说尤其如此。
   - **奖励建模的不稳定性**：奖励模型本身可能并不完美，并且训练过程中存在噪声，尤其是在对奖励函数建模时，可能会引入不一致的反馈信号，从而导致训练的不稳定。
   - **强化学习的不稳定性**：传统的RLHF采用强化学习来优化模型，这会导致训练过程非常复杂并且容易出现不稳定，尤其是在奖励信号稀疏或不一致时。

### 2. DPO 的产生
DPO的出现正是为了**简化RLHF**的流程，并且避免RLHF中的一些问题。与RLHF需要构建复杂的奖励模型并进行强化学习优化不同，DPO采用了**直接优化生成文本偏好**的方法，减少了对复杂奖励函数和人工反馈的依赖。

DPO 通过**优化模型生成文本时的偏好排序**来进行训练，而不需要像RLHF那样通过奖励模型来估计每个生成结果的价值。这种方法不仅降低了训练成本，还减少了训练过程中的不稳定性。

假设你正在训练一个**自动写作助手**，这个助手需要根据用户提供的提示生成一篇文章。你希望助手生成的文章不仅要语法正确，还要符合特定的风格和内容要求，例如需要文章具备较强的创意和吸引力。

#### 使用RLHF：
1. **数据收集**：你会让人类标注员查看助手生成的多篇文章，然后为每篇文章打分，告诉助手哪些文章更好、哪些更差。
2. **奖励模型**：你根据这些评分训练一个奖励模型，奖励模型将学习如何根据文章的得分来评估新生成文章的质量。
3. **强化学习微调**：你使用这个奖励模型对助手进行微调，让助手根据奖励信号生成更符合人类偏好的文章。

然而，RLHF 在这个过程中有几个问题：
- **高成本**：你需要大量的人类标注员来查看和评分每篇文章，工作量巨大。
- **不稳定**：奖励模型可能会受到噪声的影响，特别是人类评分的一致性问题，导致助手的微调过程可能出现不稳定。


#### 使用DPO：
1. **数据收集**：你不再依赖人工评分，而是收集一些**已知的优质文章和差文章**，并根据某些标准（比如创意、流畅性）对它们进行排序。例如，你可以根据专家评论或历史数据中用户的点击量来判断哪些文章更受欢迎。
2. **直接优化**：你直接让助手学习如何生成更符合这些排序的文章，而不需要训练一个奖励模型。DPO 会根据这些排序数据直接优化写作助手的输出，使其生成更加符合偏好的文章。

使用 DPO 的优势：
- **低成本**：你不需要依赖大量的人工标注员进行评分，只需要一小部分排序数据。
- **训练稳定**：DPO 不依赖复杂的奖励建模和强化学习过程，因此训练更加稳定。
- **高效**：由于DPO直接优化偏好排序，它能够更快地提高模型的生成能力，尤其是在目标明确的情况下。

### 总结

**DPO** 作为RLHF的一个简化版本，通过直接优化文本生成的偏好排序，避免了传统强化学习中奖励建模和人工反馈带来的不稳定性和高成本问题。DPO提供了一种更加高效、稳定且成本较低的微调方法，尤其适用于大规模语言模型的训练和优化。使得模型训练更加快速、稳定，并且减少了对人工反馈的依赖。


## 1.2 什么是DPO

**DPO（Direct Preference Optimization）**，从本质上讲，是一种**优化方法**，而不是一个传统意义上的“算法”或“流程”。它专注于通过优化文本生成的**偏好排序**来训练模型，以提高模型的输出质量。它是一种基于排序的优化思想，提升模型生成的质量。

它的核心思想是：通过直接优化生成文本的**排序偏好**，使得模型生成的文本更符合人类的需求或特定的标准，而不依赖于复杂的奖励模型或强化学习。

### 核心要点：
1. **偏好排序（Preference Ordering）**：
   - DPO 的目标是通过优化模型生成的文本与人类期望的文本之间的偏好排序来进行训练。假设有一组文本(例如上边所说已知优质文章和差文章)，DPO会根据某些标准（比如文本的流畅性、相关性等）对这些文本进行排序。然后，模型通过学习如何生成更符合这种排序的文本来进行优化。

2. **无需奖励建模**：
   - 与RLHF不同，DPO不需要一个复杂的奖励模型。RLHF中需要通过大量的人工反馈来训练一个奖励模型，而DPO通过直接利用排序数据来优化模型，无需复杂的奖励函数。

3. **优化目标**：
   - DPO的优化目标是使模型生成的文本尽可能符合给定的偏好排序。例如，如果给定一个文本集合，DPO会根据这些文本的优劣顺序来训练模型，使得模型生成的文本与偏好的顺序更加一致。


### 对比RLHF

**RLHF**可以理解成一套**流程**，而不是单纯的“算法”。RLHF中使用了多个算法（比如强化学习算法），更像是是一种基于人类反馈的**强化学习流程**，整个过程涉及多个阶段和步骤，旨在通过人类的偏好或评价来优化模型的生成行为。(简单回顾RLHF全流程)

#### 1. **数据收集：人工反馈**
   - **人类反馈**：首先，需要收集一些模型生成的输出。比如，在对话生成任务中，模型会根据给定的输入生成多个候选回答。接下来，人工评审员对这些候选回答进行评估，或者对它们进行排序。
   - **评分或排序**：人类评审员可以根据不同标准（例如，流畅性、相关性、创造性等）为生成的文本评分或排序。(这一步算流程中的核心，模型的优化最终是基于这些人工标注的反馈)

#### 2. **奖励模型训练**
   - **奖励建模**：基于人工反馈（评分或排序数据），训练一个**奖励模型**。这个奖励模型的目标是模拟人类的评判标准，它会根据生成文本的质量给出一个“奖励值”或“评分”。
   - **模型学习反馈**：奖励模型会学习如何从评分或排序数据中捕捉到“好的”输出和“差的”输出之间的区别。

#### 3. **强化学习优化**
   - **强化学习（RL）**：使用强化学习算法（如 **PPO**）来微调模型。奖励模型在这里充当“环境”，提供每个生成输出的奖励信号。
   - **策略优化**：通过强化学习，模型根据奖励信号（即奖励模型给出的评分）来调整其策略，优化输出，使得生成的文本在奖励模型的评分中能获得更高的分数。（就是通过让模型获取奖励的方式，使模型“学习”如何生成更符合人类偏好的输出）
   - **探索与利用**：强化学习过程中会平衡探索新策略和利用已有的高奖励策略，帮助模型在优化过程中避免过拟合和陷入局部最优。

#### 4. **评估与调优**
   - 在整个训练过程中，会不断评估模型生成的文本是否符合预期，并根据反馈调整训练过程。


### DPO 与 RLHF 的对比优势

| 特性                   | RLHF                                 | DPO                                  |
|------------------------|--------------------------------------|--------------------------------------|
| **人工反馈依赖**        | 需要大量的人工标注和反馈             | 通过已有的排序数据训练，无需大量人工标注 |
| **训练稳定性**          | 训练过程可能不稳定，尤其是奖励信号不一致时 | 通过直接优化偏好排序，训练过程更稳定   |
| **训练成本**            | 高，需要大量的人工参与               | 低，因为无需人工标注反馈和奖励模型   |
| **优化目标**            | 依赖复杂的奖励模型进行优化           | 直接优化生成文本的排序偏好           |
| **应用灵活性**          | 适用于多种应用，但高成本和复杂       | 适合需要低成本、快速稳定训练的场景  |

### 为什么总会与RLHF进行对比

#### 1. **DPO 和 RLHF 在核心目标上非常相似**
   - **DPO和RLHF** 都是在利用 **人类反馈** 来优化模型的行为，目标是使得生成的文本更符合人类的偏好或标准。它们的共同点是：**通过人工的评分、排序或偏好来指导模型的优化**，并最终提升生成模型的质量。
   - DPO和RLHF作为依赖 **人类反馈** 的微调方法，在优化流程和目标上有更多相似性。

#### 2. **RLHF是DPO的直接前身**
   - DPO是从RLHF中衍生出来的一个 **简化版**，它避免了RLHF中的复杂性（如奖励建模和强化学习算法），因此它的核心思想和优化流程与RLHF非常接近。
   - **DPO 的优化方式** 通过直接优化生成文本的偏好排序，简化了RLHF中需要构建奖励模型和使用强化学习的步骤。
   - 因此，**DPO的产生克服了RLHF中的复杂性**，两者在训练目标和应用场景上有很大的重叠（理解DPO与RLHF的区别和联系，能帮助更好地理解DPO的优势。）

#### 3. **DPO 和 RLHF 都注重生成任务的优化**
   - DPO和RLHF都主要应用于 **生成任务**，例如文本生成、对话生成、代码生成等任务，它们侧重于根据人类的偏好优化生成结果。这类任务中，**人类的反馈** 直接影响到模型生成的文本是否符合用户的需求，生成任务的优化本身就离不开人类的偏好信息。


#### 4. **DPO 和 RLHF 对比更有意义**
   - **相似的优化目标**：RLHF 和 DPO 都是依赖人工反馈来优化模型生成的文本，并且它们都使用了 **生成模型** 的训练框架。它们的目标是通过人类的评价来指导模型生成更符合人类偏好的文本，因此二者的对比帮助理解在使用人类反馈时，如何选择更简洁有效的优化方法。
   - **DPO 是 RLHF 的简化版本**：从RLHF到DPO的演变，正是为了克服RLHF中的复杂性，尤其是奖励建模和强化学习带来的不稳定性。因此，对比DPO和RLHF方法上的不同，能够协助了解简化训练流程、减少资源消耗并提升稳定性的方法。

### 为什么不和其他微调方法（如 LoRA、SFT、PPO 等）进行对比

   - **LoRA（Low-Rank Adaptation）**：
     LoRA 是一种 **参数高效的微调方法**，它通过在预训练模型的某些层中引入低秩矩阵来进行调整，从而以较少的计算资源和存储开销达到较好的微调效果。LoRA 主要关注如何优化模型的参数效率，减少资源消耗，并不直接涉及人类反馈或偏好排序。因此，LoRA 和 DPO 在优化的目标和方法上差异较大。
   
   - **SFT（Supervised Fine-Tuning）**：
     SFT 是一种传统的 **监督学习** 方法，它依赖标注好的训练数据，通过最小化预测和真实标签之间的损失函数来微调模型。虽然 SFT 也会优化模型的性能，但它并**没有利用人类反馈**的排序信息，且通常依赖于标注数据。因此，SFT 更多的是针对模型的准确性和泛化能力进行优化，而不是通过人类偏好来改进生成效果。

   - **PPO（Proximal Policy Optimization）**：
     PPO是一种强化学习算法，虽然它常用于 RLHF训练中，但它本身并不是一种微调方法。**PPO通过强化学习策略优化来调整模型**，而不是通过优化偏好排序（如 DPO）。PPO关注的是如何通过优化策略来增强模型的长期表现，而 DPO直接根据排序数据进行优化，侧重于模型生成质量的短期提升。
    
**LoRA、SFT、PPO等方法，优化的重点往往是模型参数效率、监督学习的准确性，或者是强化学习中的策略优化，并不专门针对生成任务中的“人类偏好”进行优化。**

# 2. DPO原理

## **2.1 原理讲解**

**Direct Preference Optimization:
Your Language Model is Secretly a Reward Model**

**直接偏好优化：
你的语言模型其实是一个奖励模型**

DPO 论文地址：https://arxiv.org/pdf/2305.18290  理解 DPO 的核心原理和理论框架 最早发布 2023年5月，最近新于2024 年7月29日

为了防止公式复杂难以理解，本次尽量会根据举例，按照数据变化方式来引导学习，对原模型推理感兴趣的同学，强烈建议查看原论文进行推导，来领略巧妙的构思！

#### **假设任务场景：AI 旅游推荐系统**

假设我们训练一个 AI，目标是为用户生成优质的旅游推荐。
- **用户输入**：如“推荐一个适合家庭旅行的地方。”
- **AI 输出**：生成多个可能的回答，如“夏威夷”、“迈阿密”、“巴黎”。
- 我们希望模型通过微调，生成更加符合用户偏好的回答。

## 首先回顾一下RLHF

**考虑RLHF的三阶段流程**

RLHF包括 **SFT（监督微调）**、**奖励建模**和**强化学习优化**三个阶段。以下是每一步的具体推导，以及输入输出的数据变化：

#### 1. SFT（监督微调）

##### 核心目标：
使用高质量的监督数据对预训练模型进行微调，得到初始模型 $ \pi_\text{SFT}(y | x) $。

##### 数据变化：
- **输入**：一个包含输入问题（prompt）和高质量答案的训练数据集，例如：
  - 输入 $ x_1 $：“推荐一个适合家庭旅行的地方。”
  - 输出 $ y_1 $：“夏威夷是一个适合家庭的海滩目的地。”
- **输出**：微调后的语言模型 $ \pi_\text{SFT}(y | x) $，它能够生成接近训练数据分布的回答。
##### 结果：
- $ \pi_\text{SFT}(y | x) $ 是一个初步具备回答能力的模型，但无法完全符合用户偏好。例如，它可能生成以下回答：
  - 回答 A（质量高但冗长）："夏威夷有很多家庭友好的活动，如浮潜和观鲸。"
  - 回答 B（简洁但不够具体）："迈阿密是一个很棒的海滩城市。"

#### 2. 偏好采样与奖励建模

##### 核心目标：
利用人类反馈，训练一个奖励模型 $ r_\phi(x, y) $，能够评估输出 $ y $ 的质量。

##### 步骤：
1. **生成回答对**：
   - 用 $ \pi_\text{SFT}(y | x) $ 生成多个候选回答。
   - 例如：
     - 回答 A："夏威夷有很多家庭友好的活动，如浮潜和观鲸。"（候选1）
     - 回答 B："迈阿密是一个很棒的海滩城市。"（候选2）

2. **人类偏好标注**：
   - 人类评审员选择更符合需求的回答：
     - 标注结果：$ y_w = \text{回答 A} $（优胜者），$ y_l = \text{回答 B} $（劣势者）。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241216215756854.png" width=100%></div>


3. **偏好建模（Bradley-Terry 模型）**：
- **建模人类偏好概率**：我们希望通过偏好数据，学习一个奖励函数 $ r(x, y) $，用来反映答案 $ y $ 的质量。
   - 假设人类偏好分布服从以下公式：
     $$
     p^*(y_1 \succ y_2 | x) = \frac{\exp(r^*(x, y_1))}{\exp(r^*(x, y_1)) + \exp(r^*(x, y_2))},
     $$

#### **公式含义**：
- 1. **输入**：
   - $ x $：问题或上下文（prompt），比如“推荐一个适合家庭旅行的地方。”
   - $ y_1 $：优胜的回答，比如“夏威夷是一个家庭友好的目的地。”
   - $ y_2 $：劣势的回答，比如“迈阿密有很多海滩。”

- 2. **偏好概率**：
   - $ p^*(y_1 \succ y_2 | x) $ 表示人类更偏好回答 $ y_1 $ 而非 $ y_2 $ 的概率。
   - $ r^*(x, y) $ 是理想奖励分数。

- 3. **行为**：
   - $ r^*(x, y) $ 的值越大，说明答案 $ y $ 的质量越高。
   - 如果 $ r^*(x, y_1) \gg r^*(x, y_2) $，偏好概率 $ p^* $ 趋近于 1，表示 $ y_1 $ 明显优于 $ y_2 $。
     其中 $ r^*(x, y) $ 是我们希望学习的奖励函数。


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241216215808067.png" width=100%></div>

4. **训练奖励模型**：
   - 用 $ r_\phi(x, y) $ 参数化奖励函数，并通过最大似然估计（MLE）优化：
     $$
     \mathcal{L}_R = -\mathbb{E}_{(x, y_w, y_l) \sim \mathcal{D}} \left[ \log \sigma(r_\phi(x, y_w) - r_\phi(x, y_l)) \right].
     $$
   - 奖励模型学习到：$ r_\phi(x, \text{回答 A}) > r_\phi(x, \text{回答 B}) $。

- 1. **逻辑函数 $ \sigma $**：
   - $ \sigma(z) = \frac{1}{1 + \exp(-z)} $，将奖励分差映射为偏好概率。

- 2. **负对数似然损失**：
   - 损失函数通过最大化 $ \sigma(r_\phi(x, y_w) - r_\phi(x, y_l)) $，学习 $ r_\phi(x, y) $ 的参数 $ \phi $。
   - **目标**：让奖励模型尽可能拟合人类偏好。

##### 数据变化：
- **输入**：偏好对数据集 $\mathcal{D} = \{(x, y_w, y_l)\}$。
- **输出**：奖励模型 $ r_\phi(x, y) $，能够为每个回答打分，例如：
  - $ r_\phi(x,y_w) = 9.0 $
  - $ r_\phi(x, y_l) = 7.5 $

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241216215822882.png" width=100%></div>

#### 3. 强化学习优化

##### 核心目标：
使用奖励模型 $ r_\phi(x, y) $ 微调策略模型 $ \pi_\theta(y | x) $，生成更符合用户偏好的回答。

##### 公式：
强化学习的目标是最大化以下优化问题：
$$
\max_{\pi_\theta} \mathbb{E}_{x \sim \mathcal{D}, y \sim \pi_\theta(y \mid x)} \left[ r_\phi(x, y) \right] - \beta \mathbb{D}_{\text{KL}} \left[ \pi_\theta(y \mid x) \parallel \pi_{\text{ref}}(y \mid x) \right]
$$

**第一项：期望奖励最大化**

$$
\max_{\pi_\theta} \mathbb{E}_{x \sim \mathcal{D}, y \sim \pi_\theta(y \mid x)} \left[ r_\phi(x, y) \right]
$$

**含义：**
- **目标**：模型 $ \pi_\theta(y | x) $ 生成的回答 $ y $，应该尽量获得高的奖励值 $ r_\phi(x, y) $。奖励值 $ r_\phi(x, y) $ 是由 **奖励模型** 训练出来的，它反映了回答 $ y $ 的质量（更符合人类偏好）。
- **这一项的任务**：强化学习要调整策略模型 $ \pi_\theta $，让它优先生成奖励更高的回答。


**第二项：KL 散度约束**

$$
-\beta \mathbb{D}_{\text{KL}} \left[ \pi_\theta(y \mid x) \parallel \pi_{\text{ref}}(y \mid x) \right]
$$

**含义：**
- **目标**：控制模型 $ \pi_\theta $ 的输出分布不要偏离 $ \pi_{\text{ref}} $（初始监督微调模型）。
- **为什么需要这项**：
  - 如果没有约束，模型可能过度优化奖励值 $ r_\phi $，生成单一模式的答案（如重复特定的高奖励答案）。
  - 这种现象称为“**模式崩塌**”，会导致生成的回答多样性丧失。
- **这一项的任务**：通过 KL 散度约束，让 $ \pi_\theta $ 的分布既能朝高奖励方向调整，又能保留 $ \pi_{\text{ref}} $ 的多样性。

这两项结合起来，可以总结为一个权衡：
1. 第一项追求更高奖励，让生成的回答更符合人类偏好。
2. 第二项用 KL 散度限制偏离，保证回答分布的多样性和稳定性。

通过调节权重参数 $ \beta $，可以控制两者的平衡：
- $ \beta $ 较小：更多关注高奖励，允许更大偏离。
- $ \beta $ 较大：更严格约束分布，偏离小，生成更稳定。

### 问题：
- RL优化过程复杂（例如使用 PPO），梯度高方差且难以调参。
- 难以保证训练的稳定性。

## 第二阶段：从 RLHF 推导到 DPO

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241216215836969.png" width=100%></div>

在 KL 散度约束下，最优的策略分布 $ \pi_r $ 具有解析形式：

$$
\pi_r(y \mid x) = \frac{1}{Z(x)} \pi_{\text{ref}}(y \mid x) \exp \left( \frac{1}{\beta} r(x, y) \right),
$$

### 含义：
1. **$\pi_{\text{ref}}(y \mid x)$**：参考模型的分布，提供基准。
2. **$\exp\left( \frac{1}{\beta} r(x, y) \right)$**：奖励加权，高奖励的答案 $ y $ 在分布中有更高的概率。
3. **$Z(x)$**：分区函数，确保 $\pi_r$ 是一个有效的概率分布。

$$
Z(x) = \sum_y \pi_{\text{ref}}(y \mid x) \exp \left( \frac{1}{\beta} r(x, y) \right)
$$

### 作用：
- 这个公式揭示了最优策略分布的结构，结合了参考模型与奖励函数。
- 但问题在于：**计算 $ Z(x) $ 非常昂贵**，使得在实际应用中难以直接优化。

DPO 的核心思想是 **跳过强化学习的复杂性**，直接利用偏好数据优化策略模型。


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241216215859294.png" width=100%></div>

为了避免直接计算分区函数 $ Z(x) $，我们对最优策略分布取对数，得到奖励函数的重新参数化形式：

$$
r(x, y) = \beta \log \frac{\pi_r(y \mid x)}{\pi_{\text{ref}}(y \mid x)} + \beta \log Z(x).
$$

### 含义：
1. **奖励函数 $ r(x, y) $**：可以用策略分布 $\pi_r$ 和参考分布 $\pi_{\text{ref}}$ 的比值表示。
2. **分区函数消除**：在偏好模型中，奖励只需要差值，因此分区函数 $ Z(x) $ 被消去。


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241216215913931.png" width=100%></div>

#### 偏好概率公式
将 $ r(x, y) $ 代入 Bradley-Terry 模型的偏好概率公式：
$$
p^*(y_w \succ y_l | x) = \frac{\exp(r(x, y_w))}{\exp(r(x, y_w)) + \exp(r(x, y_l))}.
$$

替换 $ r(x, y) $ 的定义：
$$
p^*(y_w \succ y_l | x) = \frac{\exp\left( \beta \log \frac{\pi_\theta(y_w | x)}{\pi_{\text{ref}}(y_w | x)} \right)}{\exp\left( \beta \log \frac{\pi_\theta(y_w | x)}{\pi_{\text{ref}}(y_w | x)} \right) + \exp\left( \beta \log \frac{\pi_\theta(y_l | x)}{\pi_{\text{ref}}(y_l | x)} \right)}.
$$

通过化简，得到偏好概率直接由策略分布对数几率表示：
$$
p^*(y_w \succ y_l | x) = \frac{1}{1 + \exp\left(-\beta \left[ \log \pi_\theta(y_w | x) - \log \pi_\theta(y_l | x) \right]\right)}.
$$

#### 结论
- 这一步重构将奖励优化问题转换为策略模型 $ \pi_\theta $ 的对数几率优化问题。
- 偏好概率不再需要显式奖励函数，而是直接由 $ \pi_\theta $ 定义。



### DPO 的最大似然目标

基于偏好模型，我们可以直接为参数化策略 $\pi_\theta$ 定义最大似然目标：

$$
\mathcal{L}_{\text{DPO}}(\pi_{\theta}; \pi_{\text{ref}}) = -\mathbb{E}_{(x, y_w, y_l) \sim \mathcal{D}} \left[ \log \sigma \left( \beta \log \frac{\pi_{\theta}(y_w \mid x)}{\pi_{\text{ref}}(y_w \mid x)} - \beta \log \frac{\pi_{\theta}(y_l \mid x)}{\pi_{\text{ref}}(y_l \mid x)} \right) \right].
$$

### 含义：
1. **最大似然优化目标**：
   - 优化策略模型 $\pi_\theta$，使其最大化偏好数据集中优质回答 $ y_w $ 的概率，同时抑制劣质回答 $ y_l $。
2. **奖励隐含表示**：
   - $\log \frac{\pi_\theta(y \mid x)}{\pi_{\text{ref}}(y \mid x)}$ 充当了奖励函数的角色，无需显式建模 $ r(x, y) $。
3. **避免 RL 训练循环**：
   - DPO 将奖励函数上的损失转换为策略上的损失，直接通过最大似然方法优化 $\pi_\theta$。



### DPO 损失函数

#### 目标：简化 RLHF 的优化过程
DPO 使用交叉熵损失直接优化策略模型 $ \pi_\theta $，替代复杂的 RL 优化目标。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241216215927401.png" width=100%></div>

#### DPO 损失函数
基于偏好概率 $ p^*(y_w \succ y_l | x) $，DPO 的优化目标是：
$$
\mathcal{L}_{\text{DPO}}(\pi_\theta; \pi_{\text{ref}}) = -\mathbb{E}_{(x, y_w, y_l) \sim \mathcal{D}} \left[ \log \sigma \left( \beta \log \frac{\pi_\theta(y_w \mid x)}{\pi_{\text{ref}}(y_w \mid x)} - \beta \log \frac{\pi_\theta(y_l \mid x)}{\pi_{\text{ref}}(y_l \mid x)} \right) \right].
$$

- **损失解析**：
  - $ \sigma(z) = \frac{1}{1 + \exp(-z)} $ 是逻辑函数，用于将对数几率映射为概率值。
  - 优化目标是最大化 $ y_w $ 的生成概率，最小化 $ y_l $ 的生成概率。

#### 替代 RL 优化的意义
- 不再需要显式的奖励函数 $ r(x, y) $。
- 不再需要强化学习算法（如 PPO），优化过程更简单、更稳定。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241202161220869.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241206202819274.png" width=100%></div>

## **2.2 与RLHF的区别**

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217145718097.png" width=100%></div>

### **策略**

#### **什么是 On-Policy 和 Off-Policy？**

#### On-Policy（在线策略）

- **定义**：On-policy 方法需要**使用当前策略**（即正在学习的策略）来采集数据，并使用这些数据来更新策略本身。
- **特征**：
  - 数据与当前策略 $ \pi_\theta $ 强相关。
  - 策略更新时，必须使用**当前**策略生成的数据。
- **优点**：策略与数据匹配，优化稳定性较好。
- **缺点**：每次策略更新都需要重新采样数据，数据利用效率较低。

#### Off-Policy（离线策略）

- **定义**：Off-policy 方法允许使用与当前策略**不同**的策略（如旧策略或经验池中的数据）来采集数据，用这些数据来更新当前策略。
- **特征**：
  - 数据可以来自其他策略（例如旧策略 $\pi_{\text{old}}$ 或参考策略 $\pi_{\text{ref}}$）。
  - 当前策略 $ \pi_\theta $ 仍然可以通过这些数据进行优化。
- **优点**：数据可以重复利用，提高数据效率。
- **缺点**：策略和数据不完全匹配，可能导致训练不稳定，需要额外处理（如重要性采样）。



#### RLHF和DPO微调属于哪种类型？

#### RLHF 微调：On-Policy

- **RLHF** 通常使用**当前策略**（即正在训练的策略）生成数据，并根据这些数据更新策略。具体过程：
  1. 使用当前策略 $ \pi_\theta $ 生成一批数据（如对话或文本生成结果）。
  2. 用奖励模型 $ r(x, y) $ 评估这些数据，得到奖励值。
  3. 根据奖励值更新策略 $ \pi_\theta $（如通过 PPO 算法进行更新）。
- **特点**：
  - 每次策略更新都依赖当前策略生成的新数据。
  - 这使得 RLHF 是 **on-policy** 学习方法，因为数据是由当前策略产生的。

#### DPO 微调：Off-Policy

- **DPO（Direct Preference Optimization）** 是一种 **off-policy** 方法，因为它不需要使用当前策略 $ \pi_\theta $ 重新采样数据，而是直接使用一个**离线数据集**（即人类偏好数据集）。
- DPO 微调的过程：
  1. 使用已存在的偏好数据集 $ \mathcal{D} $，其中包含人类标注的优劣答案对 $(x, y_w, y_l)$。
  2. 优化策略 $ \pi_\theta $ 以最大化优选答案 $ y_w $ 的生成概率，同时降低劣选答案 $ y_l $ 的概率。
  3. 由于数据已经固定，不需要依赖当前策略生成新数据，因此 DPO 是 **off-policy** 方法。
- **特点**：
  - 数据是静态的，不需要在线采样。
  - 策略更新过程直接基于固定的数据集进行。

我们用一个“登山者攀登山峰”的例子来形象化说明 **on-policy** 和 **off-policy** 的区别。

#### On-Policy（RLHF）：实时适应路线

想象有一位登山者正在攀登山峰，他每走一步都会评估当前的路线好不好：
1. **走一步，评估一步**：他只根据自己当前走的这条路线的体验（如地形平稳度、风景好坏）来决定下一步怎么走。
2. **数据实时更新**：他只能用自己当前走的这条路线的数据来优化接下来的路线，不考虑之前其他路线的信息。
3. **RLHF 对应**：在RLHF中，当前策略 $\pi_\theta$ 生成的数据会被立即用来优化策略本身。每次更新都需要生成新数据，类似登山者走一步，评估一步。

#### Off-Policy（DPO）：学习别人的经验

另一个登山者不自己去攀登山峰，而是收集了前人走过的各种路线的经验数据：
1. **使用固定数据**：他从前人提供的地图和路线记录中学习哪些路线好（优选）哪些不好（劣选）。
2. **策略优化**：他不需要自己重新尝试路线，而是基于这些数据来规划自己的最佳路线。
3. **DPO 对应**：DPO使用现有的偏好数据集（人类标注好的答案对）来优化策略，不需要重新采样数据。

但这些数据可能不完全适配当前任务环境。

## **数据处理与学习方式**

### Online（在线）和 Offline（离线）

**Online（在线）** 和 **Offline（离线）** 是两种不同的数据处理和学习方式，主要区别在于**数据的获取和使用方式**。

### 什么是 Online 和 Offline？
#### Online（在线学习）

- **定义**：模型在训练过程中，不断地从环境或数据源中**实时收集数据**，然后利用这些数据立即进行训练和更新。
- **特征**：
  - 数据是**动态生成**的，随着训练的进行不断采样新数据。
  - 学习过程是实时的，模型可以立即适应新数据。
- **优点**：模型可以根据实时数据进行适应性学习，适合环境动态变化的场景。
- **缺点**：每次学习都需要重新采样数据，数据利用效率较低，训练成本较高。

#### Offline（离线学习）

- **定义**：模型在训练过程中使用一个**固定的静态数据集**，数据集事先收集并存储好，训练时不会再生成新数据。
- **特征**：
  - 数据是**预先存在**的，训练过程与数据采集完全分离。
  - 训练时模型只能使用已有的数据，无法动态采样新数据。
- **优点**：数据可以反复利用，训练效率高，适合大规模数据场景。
- **缺点**：无法适应实时变化的环境，数据可能存在分布偏差。


### RLHF：Online 学习

- **RLHF 特点**：
  1. 使用当前策略 $ \pi_\theta $ 生成新的数据（例如生成新的文本）。
  2. 数据会被奖励模型 $ r(x, y) $ 评估，得到奖励值。
  3. 根据奖励值更新策略 $ \pi_\theta $。
  4. 每次策略更新后，重新采样数据，再次评估与优化。

- 总结：RLHF需要不断地使用**实时生成的数据**进行训练和更新。

### DPO：Offline 学习

- **DPO 特点**：
  1. 使用一个**固定的偏好数据集** $ \mathcal{D} $，其中包含人类标注的答案对 $(x, y_w, y_l)$（优选答案和劣选答案）。
  2. 在这个静态数据集上训练策略 $ \pi_\theta $，使模型倾向于生成优质答案 $ y_w $，并抑制劣质答案 $ y_l $。
  3. 不需要重新采样数据，也不需要实时生成新数据。
- 总结：DPO依赖**事先收集好的静态数据集**进行训练，不再与实时数据采样挂钩。

想象一个厨师正在学习如何做一道新菜（比如炒菜）

#### Online 学习（RLHF）：实时试菜法

1. **实时试菜**：厨师每做出一道菜，就亲自尝一口，评估味道好不好（类似RLHF的奖励模型评估数据）。
2. **根据反馈调整**：如果味道不好，就调整做菜的方法，比如加点盐、减少油，重新做一盘菜，再尝一尝。
3. **循环试错**：厨师通过这种实时“试菜—调整—再试”的循环，逐步找到最佳的做菜方法。

#### Offline 学习（DPO）：参考菜谱法

收集了很多名厨的菜谱和评价记录：

1. **使用固定数据**：这些菜谱详细记录了各种做菜方法，以及哪些菜式被评为最好（人类偏好数据）。
2. **学习菜谱**：厨师仔细研究菜谱，学习最优的做菜方法，直接模仿名厨的优质菜式，避免重复试错。
3. **直接优化**：厨师可以根据固定的菜谱数据，优化自己的做菜技巧，而不需要再做菜试验。

### 对比

| **学习类型** | **数据来源**           | **数据使用方式**     | **RLHF 与 DPO**    | **示例**                 |
|--------------|------------------------|----------------------|-------------------|--------------------------|
| **Online**   | 实时生成数据           | 动态采样，不断更新   | RLHF 微调         | 厨师做菜后实时试味道     |
| **Offline**  | 固定数据集（事先收集） | 数据静态，可重复利用 | DPO 微调          | 厨师研究固定菜谱和记录   |

DPO 的优势在于避免了复杂的强化学习过程，数据利用效率更高，非常适合在固定偏好数据集上高效优化语言模型。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217145937860.png" width=100%></div>

# 3. DPO微调实操

- **step1 准备数据集**

外部数据集llama_factory官网： https://github.com/hiyouga/LLaMA-Factory/blob/main/README_zh.md#%E6%95%B0%E6%8D%AE%E9%9B%86

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217215629248.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217215734612.png" width=100%></div>

也可以直接huggingface搜索数据

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217215605378.png" width=100%></div>

选择数据  https://huggingface.co/datasets/Starlento/DPO-En-Zh-20k-handbook?row=15

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217215502442.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217220559214.png" width=100%></div>

**国内modelscope社区**：https://www.modelscope.cn/datasets?page=1&query=DPO-En-Zh-20k&sort=latest 

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217220525670.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217220559214.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217220640663.png" width=100%></div>

- **step 2 数据处理**

数据上传并校验

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217225025687.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217225145227.png" width=100%></div>

根据个人喜好重命名  mv dpo_zh.json dpo_zh_1218.json

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217225251142.png" width=100%></div>

查验parquet文件格式内容
```python
import pyarrow.parquet as pq

# 读取 parquet 文件
file_path = '/home/data/muyan/download_data/train-00000-of-00001.parquet'
table = pq.read_table(file_path)
df = table.to_pandas()

# 查看前 10 条记录
print("First 10 rows:")
print(df.head(10))
```


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217225927231.png" width=100%></div>

```python
# 查看第三条记录的详细信息
print("\nDetailed view of third row (index 2):")
print(df.iloc[2])
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217230024499.png" width=100%></div>

查看指定条全部内容
```python
import pandas as pd
# 读取 parquet 文件
df = pd.read_parquet(file_path)
# 查看第三条记录的详细信息
print("\nDetailed view of third row (index 2):")
with pd.option_context('display.max_colwidth', None, 'display.expand_frame_repr', False):
​    print(df.iloc[2])
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217230715842.png" width=100%></div>

```json
偏好数据集需要在 chosen 列中提供更优的回答，并在 rejected 列中提供更差的回答，在一轮问答中其格式如下：
[
  {
    "instruction": "人类指令（必填）",
    "input": "人类输入（选填）",
    "chosen": "优质回答（必填）",
    "rejected": "劣质回答（必填）"
  }
]
```


代码文件：选取中文的前5000条  convert_parquet_zh_5000.py
```python
import pandas as pd
import json
import sys

def convert_to_alpaca_format(input_file_path, output_file_path):
    # 读取 parquet 文件
    df = pd.read_parquet(input_file_path)
    
    # 过滤出 language 为 zh 的记录
    filtered_df = df[df['language'] == 'zh']
    
    # 获取前 5000 条记录
    first_5000_records = filtered_df.head(5000)
    
    # 创建新的列表来存储 Alpaca 格式的数据
    alpaca_format = []
    
    for _, row in first_5000_records.iterrows():
        # 提取 prompt 作为 instruction
        instruction = row['prompt'].replace('\n', ' ')
        
        # 提取 chosen 和 rejected 的内容
        chosen_content = ""
        rejected_content = ""
        
        for item in row['chosen']:
            if item['role'] == 'assistant':
                chosen_content = item['content'].replace('\n', ' ')
        
        for item in row['rejected']:
            if item['role'] == 'assistant':
                rejected_content = item['content'].replace('\n', ' ')
        
        # 构建新的 Alpaca 格式数据
        alpaca_format.append({
            "instruction": instruction,
            "input": "",  # 假设没有人类输入字段，如果有请根据实际情况修改
            "chosen": chosen_content,
            "rejected": rejected_content
        })
    
    # 打印前几条记录以检查结果
    print(alpaca_format[:5])
    
    # 将结果保存为 JSON 文件（可选）
    with open(output_file_path, 'w', encoding='utf-8') as file:
        json.dump(alpaca_format, file, indent=4, ensure_ascii=False)

if __name__ == "__main__":
    if len(sys.argv) != 3:
        print("Usage: python script.py <input_file_path> <output_file_path>")
        sys.exit(1)

    input_file_path = sys.argv[1]
    output_file_path = sys.argv[2]

    convert_to_alpaca_format(input_file_path, output_file_path)
```

python convert_parquet_zh_5000.py train-00000-of-00001.parquet dpo_convert_parquet_zh_5000.json

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217231823729.png" width=100%></div>

tail -n 100 dpo_convert_parquet_zh_5000.json 手动验证数据格式

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217231901493.png" width=100%></div>

- **step3 注册数据**

cp dpo_convert_parquet_zh_5000.json ../LLaMA-Factory/data/

cp dpo_zh_1218.json ../LLaMA-Factory/data/

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217232056803.png" width=100%></div>

注册内容 
```json
  "dpo_convert_parquet_zh_5000": {
    "file_name": "dpo_convert_parquet_zh_5000.json",
    "ranking": true,
    "columns": {
      "prompt": "instruction",
      "chosen": "chosen",
      "rejected": "rejected"
    }
  },
  "dpo_zh_1218": {
    "file_name": "dpo_zh_1218.json",
    "ranking": true,
    "formatting": "sharegpt",
    "columns": {
      "messages": "conversations",
      "chosen": "chosen",
      "rejected": "rejected"
    }
  },
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217232908251.png" width=100%></div>

界面验证数据

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218115141904.png" width=100%></div>

下载模型步骤之前课上多次展示本次略过

- **step3 开始模型微调**

方式一：界面启动

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218120056300.png" width=100%></div>

```json
llamafactory-cli train \
    --stage dpo \ //类型为DPO
    --do_train True \
    --model_name_or_path /home/data/muyan/model/Qwen/Qwen2.5-0.5B-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir data \
    --dataset dpo_convert_parquet_zh_5000 \
    --cutoff_len 2048 \
    --learning_rate 5e-05 \
    --num_train_epochs 3.0 \
    --max_samples 100000 \
    --per_device_train_batch_size 2 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps 100 \
    --warmup_steps 0 \
    --packing False \
    --report_to none \
    --output_dir saves/Qwen2.5-0.5B-Instruct/lora/train_2024-12-18-11-50-58 \
    --bf16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --optim adamw_torch \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0 \
    --lora_target all \
    --pref_beta 0.1 \ // 偏好损失中的 beta 参数。
    --pref_ftx 0 \ //DPO 训练中的 sft loss 系数。
    --pref_loss sigmoid //DPO 训练中使用的偏好损失类型。可选值为：sigmoid, hinge, ipo, kto_pair, orpo, simpo。
```

**场景：学生学习作文评分**
- 假设你是一名学生（微调模型 $ \pi_\theta $），正在学习如何写一篇好作文。
- 老师（参考模型 $ \pi_{\text{ref}} $）给出了两篇作文：
  - **好作文 $ y_w $：** 被标为优选答案。
  - **坏作文 $ y_l $：** 被标为劣选答案。

 **$\beta$ 的不同值对应的场景：**
1. **$\beta$ 较大：**
   - 老师更严格：作文评分的分数差异被放大，学生对优选作文 $ y_w $ 的学习更强烈，同时更加远离坏作文 $ y_l $。
   - **结果：** 学生进步更快，但如果老师的评分标准有偏差，学生可能会过度调整，偏离其他写作风格（不够稳定）。

2. **$\beta$ 较小：**
   - 老师更宽容：评分差异较小，学生会温和地调整自己，逐步学习老师的标准。
   - **结果：** 学生进步较慢，但学习过程更加稳定，不容易出现极端现象。


- **SFT Loss**：相当于语文老师指导学生的基本写作规则（比如语法、句式规范）。
- **DPO Loss**：相当于作文评审老师根据作文内容打分（好作文和坏作文的对比）。

 **SFT Loss 系数的调节：**
- **$\lambda_{\text{SFT}}$ 较大：**
  - 学生主要关注写作基本规则，作文逻辑和语法不会出错，但内容可能不够生动。
- **$\lambda_{\text{SFT}}$ 较小：**
  - 学生更倾向于迎合评审老师的评分标准，但可能在写作规范上出现偏差。


| **损失函数**  | **适用场景**                                                                                 | **优缺点**                                       |
|---------------|---------------------------------------------------------------------------------------------|------------------------------------------------|
| **sigmoid**   | 偏好差异不大且希望稳定收敛的场景                                                             | 稳定，但更新速度慢                              |
| **hinge**     | 偏好差异较大，且需要明确区分优劣答案的场景                                                   | 梯度不连续，可能导致优化不平滑                  |
| **ipo**       | 偏好建模经典场景，数据质量较高                                                               | 收敛性好，但对噪声敏感                          |
| **kto_pair**  | 偏好分布复杂、非线性的场景                                                                   | 捕捉复杂关系，但计算复杂度高                    |
| **orpo**      | 偏好数据有噪声，或者标注质量较差的场景                                                       | 对噪声鲁棒性强，但复杂度高                      |
| **simpo**     | 偏好分布简单或需要高效训练的场景                                                             | 计算高效，但对复杂关系的捕捉能力较弱            |

方式二 ：后台运行(推荐)

创建config路径可以点击保存训练参数后，自行在相对路径下创建config文件夹。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218122350525.png" width=100%></div>

将文件放到config/ 路径下 命名 2024_12_17_23_32_50_q0_5.yaml
```json
stage: dpo
do_train: True
model_name_or_path: /home/data/muyan/model/Qwen/Qwen2.5-0.5B-Instruct
preprocessing_num_workers: 16
finetuning_type: lora
template: qwen
flash_attn: auto
dataset_dir: data
dataset: dpo_convert_parquet_zh_5000
cutoff_len: 2048
learning_rate: 5e-05
num_train_epochs: 30.0
max_samples: 200000
per_device_train_batch_size: 1
gradient_accumulation_steps: 8
lr_scheduler_type: cosine
max_grad_norm: 1.0
logging_steps: 5
save_steps: 100
warmup_steps: 0
packing: False
report_to: none
output_dir: saves/Qwen2.5-0.5B-Instruct/lora/train_2024-12-17-23-32-50
fp16: True
plot_loss: True
ddp_timeout: 180000000
optim: adamw_torch
lora_rank: 8
lora_alpha: 16
lora_dropout: 0
lora_target: all
pref_beta: 0.1
pref_ftx: 0
pref_loss: sigmoid
```

/home/data/muyan/LLaMA-Factory路径下执行命令(config 相对路径） nohup llamafactory-cli train config/2024_12_17_23_32_50_q0_5.yaml >2024_12_17_23_32_50_q0_5.log 2>&1 &

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217233547350.png" width=100%></div>

监控日志信息 tail -f 2024_12_17_23_32_50_q0_5.log

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217233732128.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217233747036.png" width=100%></div>

监控资源使用情况：watch -n 1 nvidia-smi

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241217233834392.png" width=100%></div>

训练完成下载过程图

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218121435076.png" width=100%></div>

自行指定ref_model

```json
stage: dpo
do_train: True
model_name_or_path: /home/data/muyan/model/AI-ModelScope/Llama-3.2-1B-Instruct
preprocessing_num_workers: 16
finetuning_type: lora
template: llama3
flash_attn: auto
dataset_dir: data
dataset: dpo_zh_demo
cutoff_len: 2048
learning_rate: 5.0e-05
num_train_epochs: 10.0
max_samples: 100000
per_device_train_batch_size: 1
gradient_accumulation_steps: 8
lr_scheduler_type: cosine
max_grad_norm: 1.0
logging_steps: 5
save_steps: 100
warmup_steps: 0
packing: False
report_to: none
output_dir: saves/Llama-3.2-1B-Instruct/lora/train_2024-12-17-22-54-30
bf16: True
plot_loss: True
ddp_timeout: 180000000
optim: adamw_torch
lora_rank: 8
lora_alpha: 16
lora_dropout: 0
lora_target: all
pref_beta: 0.1
pref_ftx: 0
pref_loss: sigmoid
ref_model: /home/data/muyan/model/Qwen/Qwen2.5-0.5B-Instruct
```
或者 ref_model_adapters str 参考模型的适配器路径。 

ref_model_quantization_bit(可选) int类型 参考模型的量化位数，支持 4 位或 8 位量化。 

# 4. 效果及验证

```json
Qwen2.5-0.5B-Instruct
dataset: dpo_convert_parquet_zh_5000
learning_rate: 5.0e-05
num_train_epochs: 10.0
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218144201897.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218144238151.png" width=100%></div>

效果验证：

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218145312375.png" width=100%></div>

原生：

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218144827768.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218145016685.png" width=100%></div>

微调后：

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218145222263.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218145252632.png" width=100%></div>

也可以尝试使用llama_factory自带数据集

Qwen2.5-1.5B-Instruct 数据集:dpo_zh_demo     epochs:30

阅读下面的段落并回答问题或按照指示操作：FGA如何利用其政治关系在联邦层面推进其议程？ 段落：一个位于佛罗里达州的鲜为人知的政策团体，与众议院议长保罗·D·瑞安（R-Wis.）以及两位最保守的共和党州长有联系，已成为食品券计划中新工作要求辩论中最响亮和最有说服力的声音之一。 政府责任基金会——由缅因州州长保罗·勒佩奇（R）的前顾问领导——在将其想法带到华盛顿之前，在堪萨斯州、密西西比州和其他州测试福利变革六年。 上周五，当众议院就一项旨在对食品券接受者的工作要求进行全面改革的农业法案进行投票时，这些想法——以及FGA利用州政治关系和政策一句话成为共和党战略讨论中的一个焦点——受到关注。 由于与移民问题无关的共和党内部斗争，该法案最终未能通过。但政治观察人士表示，提议对食品券进行改革的举措证明了FGA在关键共和党圈子中日益增长的影响力。 上周五涉及的工作要求将从根本上改变许多成年接受者的食品券计划。根据该计划，18至60岁之间的大多数有劳动能力的人必须每周工作或参加州培训计划20小时才能获得福利。 众议院共和党人——包括瑞安，他是在2016年通过堪萨斯州州长山姆·布朗巴克介绍给该团体的——一再提供FGA的分析作为大多数美国人支持福利计划严格工作规则以及这些规则提高收入和就业的证据。 FGA以及更加成熟的智库，如传统基金会和美国企业研究所，长期以来一直主张将福利接受者的福利与就业挂钩，福利接受者离开贫困的速度更快。 但与老一辈不同，FGA——以其低调的国家形象、26名远程员工和2017年约700万美元的年度捐款——直到现在一直致力于州政策，首席执行官塔伦·布拉格登表示。 这种关注在2016年发生了变化，当时共和党人占据了白宫、众议院和参议院，布拉格登在布朗巴克的要求下与瑞安会面，布朗巴克是瑞安的前上司。FGA曾与布朗巴克合作加强堪萨斯州的食品券工作要求，这是瑞安支持的事业。 2016年5月与布拉格登会面以及随后的对话中，瑞安表达了对FGA工作和研究的兴趣，布拉格登说。2017年1月，FGA聘请了其第一位联邦游说者。一年后，布拉格登向众议院共和党领导人做了一个关于“如何成功地将人们从福利转移到工作”项目的演示，例如提供食品福利的SNAP计划。 FGA过去通过推动禁止州寻求食品券要求豁免的法案，以及倡导医疗补助工作要求并反对医疗补助扩展来实现这一目标。 根据税务文件，自由主义和保守派慈善基金，包括捐赠者信托基金和布拉德利基金会，提供了这些努力约四分之一的资金，而其余资金来自匿名个人捐赠者。 布拉格登的团队包括堪萨斯州和德克萨斯州保守派政府的退伍军人，以及布拉格登在右翼缅因州传统政策中心担任负责人以及后来担任勒佩奇顾问时认识的改革福利的倡导者。 在缅因州议会及周边工作了十年后，布拉格登和一群年轻的保守派赢得了对立方的声誉。在勒佩奇的领导下，缅因州对现金福利福利设定了五年的时间限制，并恢复了有劳动能力的成年人的全面SNAP工作要求。 离开缅因州后，布拉格登试图推广该州的许多政策。FGA制定了模型州立法，被称为HOPE法案，禁止州寻求SNAP工作规则的联邦豁免。自2016年以来，该法案已在16个州提出，并在堪萨斯州、密西西比州和威斯康星州通过。FGA表示该法案的部分内容已在28个州采纳。 FGA还将积极社会服务管理者推荐到共和党政府中的职位，这是其在“FGA人才库”上宣传的服务，作为其改革福利工作的一部分。 它还不断推出信息图表、民意调查和第一人称视频来推广其政策，其中许多宣传2016年FGA在堪萨斯州进行的一项研究，声称恢复SNAP工作要求促使数千名失业的堪萨斯人找到工作，并使他们的平均收入翻了一番。 尽管该研究得到共和党人的大力宣传，但受到自由派和保守派经济学家的批评，因为其数据选择性。芝加哥大学的应用微观经济学家杰弗里·格罗格表示，该论文存在一些问题，其中之一是只报告了失去福利后找到工作的前食品券接受者的结果。 格罗格还表示，该论文未能建立福利政策变化与接受者结果之间的直接关系。他说，大多数福利接受者无论是否有工作要求，都会找到工作并离开该计划。 他还表示，该论文与科学文献相矛盾，后者主要发现这些规则并不会大大改善接受者的收入，甚至可能会对他们造成伤害。 除了堪萨斯州的研究外，众议院共和党人还分发了FGA对三年前美国农业部数据的分析，该数据按州列出了失业的有劳动能力的SNAP接受者的数量。《华盛顿邮报》对更近期的美国农业部数据进行的审查表明，目前的食品券接受者人数低于FGA所声称的数字。 该团体还传播了民意调查数据，表明几乎所有美国人都支持更严格的工作要求，但像彼得·杰曼尼斯这样的观察人士批评这些数字过于简化公众意见和现有证据。 在一次采访中，布拉格登为他的团队的分析辩护，称其他研究人员可以使用“不同的方法”。他说，FGA的民意调查结果与其他团体进行的调查结果一致。而且FGA去年准备了其州级分析，当时最新数据尚未公布。 布拉格登表示，他“为自己的团队的研究感到自豪”，并推动让尽可能多的立法者了解。FGA专门制作精美的政策一句话，这些一句话以其彩色标题和引人注目的图形，似乎受益于布拉格登的兄弟特雷弗，他是一位行为科学家，曾与FGA合作，专门从事保守派事业的“有说服力的品牌推广”。 特雷弗·布拉格登的一名员工萨姆·阿道夫森目前被借调到该基金会，他担任“高级研究员”一职，并最近在众议院监督和政府改革委员会就SNAP进行了证词。 专家表示，FGA在政策信息传播和营销方面的强调使其与传统智库有所区别。 在农业法案上，FGA大力支持共和党计划，走访了议员办公室，并不断发布广泛传播的旨在转移自由派和茶党批评的“一句话”。 共和党领导层向众议院共和党人发送的四月战略电子邮件包括四张一句话，以及四个FGA视频和50个展示FGA的SNAP接受者估计的图形。华盛顿州共和党议员会议主席凯西·麦克莫里斯·罗杰斯（R-Wash.）在周三发表了一篇措辞强烈的计划辩护，专门引用了FGA的研究。一天前，瑞安出现在一段FGA视频中，赞扬工作的价值。 FGA专注于向立法者提供数据，FGA的倡导机构机会解决方案项目的游说者克里斯蒂娜·拉斯穆森表示。她本周从伊利诺伊州的家中驱车12小时，以回答共和党人关于工作要求计划的问题。 周一，拉斯穆森对该措施能够通过表示乐观。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241213202716704.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241213202748540.png" width=100%></div>

微调后

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241213203123325.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241213203321229.png" width=100%></div>

LLama-3.2-1B  ref_model:Qwen_2.5_1.5  

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218151018973.png" width=100%></div>

# 5. 源码解析

查找命令执行文件 which llamafactory-cli

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204143839942.png" width=100%></div>

查看内容 cat /root/anaconda3/envs/llama_factory/bin/llamafactory-cli

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204143925123.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204144053237.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241204144109889.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218151908106.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218152120146.png" width=100%></div>

ref_model

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218153209391.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218153351143.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218153527008.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218153646035.png" width=100%></div>

def  create_ref_model

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218154004366.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218154601337.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218154446494.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218154728427.png" width=100%></div>

```json
 ref_model (`PreTrainedModelWrapper`):
            Hugging Face transformer model with a casual language modelling head. Used for implicit reward computation and loss. If no
            reference model is provided, the trainer will create a reference model with the same architecture as the model to be optimized.

 ref_model (`PreTrainedModelWrapper`)：
带有随意语言建模头的 Hugging Face Transformer 模型。用于隐式奖励计算和损失。如果没有提供
参考模型，训练器将创建一个与要优化的模型具有相同架构的参考模型。
```

原生代码有声明，但是被子类重写

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218155512629.png" width=100%></div>

计算参考模型的对数概率。

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218155914458.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218160101753.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218160348183.png" width=100%></div>

```json
寻找思路：在损失值计算中去找
涉及到类为：CustomDPOTrainer、Trainer、DPOTrainer
涉及到方法：compute_reference_log_probs、get_batch_loss_metrics、compute_loss 、training_step、_inner_training_loop 
```

# 总结

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20241218162803961.png" width=100%></div>